In [19]:
#All imports go here
from keras.models import Sequential
from keras.layers import Convolution2D, Flatten,Dense, Lambda, MaxPooling2D
from keras.regularizers import l2

import cv2
import numpy as np
import pandas as pd

In [20]:
# Load data
DATA_DIR = './data/'
LOGFILE_PATH = DATA_DIR + 'driving_log.csv'
IMG_DIR = DATA_DIR + 'IMG/'

logs = pd.read_csv(LOGFILE_PATH,sep=',',header=None)


In [21]:
# Load image data.
CORRECTION_FACTOR = 0.2 # Steering correction factor for left and right images.
img_paths = list(logs[0]) + list(logs[1]) + list(logs[2])

image_data = []

for path in img_paths:
    file_name = path.split('/')[-1]
    img = cv2.imread(IMG_DIR+file_name)
    image_data.append(img)

image_data = np.array(image_data)


#Load steering angles
steering_angles  = np.array(list(logs[3]))
steering_angles  = np.hstack((steering_angles, steering_angles+CORRECTION_FACTOR, steering_angles-CORRECTION_FACTOR))




In [22]:
# Augment data by flipping images and steering angles.
image_data_flipped = np.empty_like(image_data)

for i in range(len(image_data)):
    image_data_flipped[i] = np.fliplr(image_data[i])
    
steering_angles_flipped = -steering_angles


image_data = np.vstack((image_data,image_data))
steering_angles = np.hstack((steering_angles,steering_angles_flipped))


In [23]:
# Build and save model.
model = Sequential()
model.add(Lambda(lambda x: x/255 - 0.5,input_shape=image_data.shape[1:]))
model.add(Convolution2D(6,5,5,activation='relu',W_regularizer=l2(),b_regularizer=l2()))
model.add(MaxPooling2D())
model.add(Convolution2D(16,5,5,activation='relu',W_regularizer=l2(),b_regularizer=l2()))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120,activation='relu',W_regularizer=l2(0.5),b_regularizer=l2(0.5)))
model.add(Dense(84,activation='relu',W_regularizer=l2(0.5),b_regularizer=l2(0.5)))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')

model.fit(image_data, steering_angles,  nb_epoch=3,validation_split=0.2, shuffle=True,verbose=2)

model.save('lenet5.3.h5')

Train on 19488 samples, validate on 4872 samples
Epoch 1/3
47s - loss: 5.0614 - val_loss: 0.0849
Epoch 2/3
46s - loss: 0.0446 - val_loss: 0.0860
Epoch 3/3
46s - loss: 0.0447 - val_loss: 0.0888
